In [1]:
import csv
import re
import numpy as np
import time
reader = csv.reader(open('/Users/mitalibhiwande/Desktop/DIC/Lab4/input/Activity2/new_lemmatizer.csv'))

# reads the lemmatizer.csv and maps the word and corresponding lemmas into a dictionary.

lemmadict = {}
for row in reader:
    #row=split(",", row)
    key = row[0]
    if key in lemmadict:
        pass
    lemmadict[key] = ' '.join(row[1:]).split()#filter(None,row[1:])


In [2]:
sc=SparkContext.getOrCreate()

In [3]:
# Reads all the input tess files present in the input directory

raw_rdd = sc.textFile('/Users/mitalibhiwande/Desktop/DIC/Lab4/input/Activity2/*.tess')

# n-grams ( n =2 )

In [4]:
#mapper function for 2grams

def sparkmapper2gram(line):
    
    results=dict()
    #print(line)
    fields = line.split(">")
    #print(fields[0])
    location=fields[0]+">"
    #print(location)
    tokens=re.sub('[^a-zA-Z0-9\\s]',"",(fields[1]))
    tokens=re.sub('j','i',tokens)
    tokens=re.sub('v','u',tokens)
    
    tokens=tokens.strip().split(" ")
    
    #print(tokens[1])
    for i in range(0,(len(tokens)-2)):
        firstlist=list()
        first=tokens[i].lower()
    
        if first in lemmadict:
            firstlist=lemmadict.get(first)
        else:
            firstlist=first
        for j in range(i+1, len(tokens)-1):
            
    #print(firstlist)
    
            second=tokens[j].lower()
            secondlist=list()
            if second in lemmadict:
                secondlist=lemmadict.get(second)
            else:
                secondlist=second
            
            for x in firstlist:
                for y in secondlist:
                    results[x,y]=location

    return results
    

In [5]:
# Maps all the documents line by line then reduces the values for the same key by appending them(locations)


start_time = time.clock()
rows=sc.parallelize(raw_rdd.take(raw_rdd.count())).map(sparkmapper2gram).flatMap(lambda x: x.items())
#rows=sc.flatMap(lambda x: x.items())
cooccured=rows.reduceByKey(lambda pair,locate:pair+locate)
print(time.clock() - start_time, "seconds")

7.170800000000001 seconds


# n-grams( n=3 )

In [6]:
def sparkmapper3gram(line):
    #print("im here")
    results=dict()
    #print(line)
    fields = line.split(">")
    #print(fields[0])
    location=fields[0]+">"
    #print(location)
    tokens=re.sub('[^a-zA-Z0-9\\s]',"",(fields[1]))
    tokens=re.sub('j','i',tokens)
    tokens=re.sub('v','u',tokens)
    
    tokens=tokens.strip().split(" ")
    
    #print(tokens[1])
    for i in range(0,(len(tokens)-3)):
        firstlist=list()
        first=tokens[i].lower()
    
        if first in lemmadict:
            firstlist=lemmadict.get(first)
        else:
            firstlist=first
    #print(firstlist)
        for j in range(i+1,len(tokens)-2):
            
            second=tokens[j].lower()
            secondlist=list()
            if second in lemmadict:
                secondlist=lemmadict.get(second)
            else:
                secondlist=second
            for k in range(j+1,len(tokens)-1):
                
                third=tokens[k].lower()
                thirdlist=list()
                if third in lemmadict:
                    thirdlist=lemmadict.get(third)
                else:
                    thirdlist=third
            
                for x in firstlist:
                    for y in secondlist:
                        for z in thirdlist:
                            results[x,y,z]=location
                

    return results
    

In [7]:
start_time = time.clock()
rows3grams=sc.parallelize(raw_rdd.take(raw_rdd.count())).map(sparkmapper3gram).flatMap(lambda x: x.items())
#rows3grams=sc.parallelize(rows3grams)
cooccured3grams=rows3grams.reduceByKey(lambda pair,locate:pair+locate)
print(time.clock() - start_time, "seconds")

7.806895000000001 seconds


# Write and display output

In [8]:
cooccured.saveAsTextFile("2gramFinalOutput")

In [9]:
cooccured3grams.saveAsTextFile("3gramFinalOutput")

In [10]:
dfWithSchema2grams = spark.createDataFrame(cooccured).toDF("n=2", "location")

In [11]:
dfWithSchema3grams = spark.createDataFrame(cooccured3grams).toDF("n=3", "location")

In [13]:
dfWithSchema2grams.show()
dfWithSchema3grams.show()

+------------------+--------------------+
|               n=2|            location|
+------------------+--------------------+
|      [bellum,per]|<luc. 1.1><luc. 1...|
|      [bellus,per]|<luc. 1.1><luc. 1...|
| [emathius,multus]|          <luc. 1.1>|
|    [emathius,qui]|          <luc. 1.1>|
|      [multus,qui]|<luc. 1.1><luc. 1...|
|    [quis,ciuilis]|<luc. 1.1><luc. 1...|
|        [ius,data]|          <luc. 1.2>|
|       [ius,datus]|          <luc. 1.2>|
|   [scelerus,cano]|          <luc. 1.2>|
|     [scelus,cano]|          <luc. 1.2>|
|[scelerus,populus]|          <luc. 1.2>|
|  [scelus,populus]|          <luc. 1.2>|
|    [cano,populus]|          <luc. 1.2>|
|     [in,conuerto]|          <luc. 1.3>|
|    [suus,uictrix]|          <luc. 1.3>|
|    [suus,uiscera]|          <luc. 1.3>|
| [uictrix,uiscera]|          <luc. 1.3>|
|  [cognatus,rumpo]|          <luc. 1.4>|
|  [cognata,foedus]|          <luc. 1.4>|
|     [acies,foedo]|          <luc. 1.4>|
+------------------+--------------